In [13]:
import numpy as np
import pandas as pd
import requests

In [14]:
def query(ndays):
    n_days = ndays
    query = 'binderhub_launch_time_seconds_count{}{}'
    query_days = "[{}d]".format(n_days)
    query_selectors = "{status='success'}"
    query = query.format(query_selectors, query_days)
    resp = requests.get('https://notebooks.gesis.org/prometheus/api/v1/query', params={'query': query})
    data = resp.json()['data']['result']
    return data


In [15]:
def makedf(data):
    df = {'name': [], 'org': [], 'provider': [], 'builds': [], 'url': []}
    for container in data:
        if 'repo' not in container['metric']:
            continue
        url = container['metric']['repo']
        pref, org, repo = container['metric']['repo'].rsplit('/', 2)
        provider = pref.replace('https://', '')
        counts = [int(ii[1]) for ii in container['values']]
        builds = np.max(counts)
        df['name'].append(repo)
        df['org'].append(org)
        df['provider'].append(provider)
        df['builds'].append(builds)
        df['url'].append(url)
    df = pd.DataFrame(df)
    df = df.drop_duplicates(['name'])
    df = df.groupby(['name', 'org', 'provider','url']).sum().reset_index().sort_values('builds', ascending=False)
    df['log_builds'] = df['builds'].apply(np.log)
    return df

In [16]:
def make_clickable(val):
    return '<a href="{}">{}</a>'.format(val, val)

# The most popular repositories of the day 

In [17]:
data=query(1)
df=makedf(data)
df.style.format({'url': make_clickable})

,name,org,provider,url,builds,log_builds
11,workshop_girls_day,gesiscss,github.com,https://github.com/gesiscss/workshop_girls_day,12,2.48491
1,PythonDataScienceHandbook,gesiscss,github.com,https://github.com/gesiscss/PythonDataScienceHandbook,8,2.07944
2,flow,gesiscss,github.com,https://github.com/gesiscss/flow,5,1.60944
5,jupyterlab-demo,jupyterlab,github.com,https://github.com/jupyterlab/jupyterlab-demo,4,1.38629
10,tidyverse,tidyverse,github.com,https://github.com/tidyverse/tidyverse,4,1.38629
4,jupyterlab,binder-examples,github.com,https://github.com/binder-examples/jupyterlab,3,1.09861
6,ligo-binder,minrk,github.com,https://github.com/minrk/ligo-binder,3,1.09861
8,requirements,binder-examples,github.com,https://github.com/binder-examples/requirements,3,1.09861
9,stmdemo,arnim,github.com,https://github.com/arnim/stmdemo,3,1.09861
0,CSSproject,nadjalc,github.com,https://github.com/nadjalc/CSSproject,2,0.693147


# The most popular repositories of the last week

In [18]:
data=query(7)
df=makedf(data)
df.style.format({'url': make_clickable})

,name,org,provider,url,builds,log_builds
11,workshop_girls_day,gesiscss,github.com,https://github.com/gesiscss/workshop_girls_day,12,2.48491
1,PythonDataScienceHandbook,gesiscss,github.com,https://github.com/gesiscss/PythonDataScienceHandbook,8,2.07944
2,flow,gesiscss,github.com,https://github.com/gesiscss/flow,5,1.60944
5,jupyterlab-demo,jupyterlab,github.com,https://github.com/jupyterlab/jupyterlab-demo,4,1.38629
10,tidyverse,tidyverse,github.com,https://github.com/tidyverse/tidyverse,4,1.38629
4,jupyterlab,binder-examples,github.com,https://github.com/binder-examples/jupyterlab,3,1.09861
6,ligo-binder,minrk,github.com,https://github.com/minrk/ligo-binder,3,1.09861
8,requirements,binder-examples,github.com,https://github.com/binder-examples/requirements,3,1.09861
9,stmdemo,arnim,github.com,https://github.com/arnim/stmdemo,3,1.09861
0,CSSproject,nadjalc,github.com,https://github.com/nadjalc/CSSproject,2,0.693147


# The most popular repositories of the last 30 days

In [19]:
data=query(30)
df=makedf(data)
df.style.format({'url': make_clickable})

,name,org,provider,url,builds,log_builds
33,workshop_girls_day,gesiscss,github.com,https://github.com/gesiscss/workshop_girls_day,69,4.23411
14,flow,gesiscss,github.com,https://github.com/gesiscss/flow,25,3.21888
4,PythonDataScienceHandbook,gesiscss,github.com,https://github.com/gesiscss/PythonDataScienceHandbook,24,3.17805
7,YoutubeScrapingEmojis,JuKo007,github.com,https://github.com/JuKo007/YoutubeScrapingEmojis,18,2.89037
3,GitHub_traffic_crawler,gesiscss,github.com,https://github.com/gesiscss/GitHub_traffic_crawler,12,2.48491
27,requirements,binder-examples,github.com,https://github.com/binder-examples/requirements,9,2.19722
28,scientific-python-lectures,jrjohansson,github.com,https://github.com/jrjohansson/scientific-python-lectures,9,2.19722
20,ligo-binder,minrk,github.com,https://github.com/minrk/ligo-binder,7,1.94591
11,conda,binder-examples,github.com,https://github.com/binder-examples/conda,7,1.94591
5,RStan-Binder,arnim,github.com,https://github.com/arnim/RStan-Binder,6,1.79176


# The most popular repositories of last 60 days

In [20]:
data=query(30)
df=makedf(data)
df.style.format({'url': make_clickable})

,name,org,provider,url,builds,log_builds
33,workshop_girls_day,gesiscss,github.com,https://github.com/gesiscss/workshop_girls_day,69,4.23411
14,flow,gesiscss,github.com,https://github.com/gesiscss/flow,25,3.21888
4,PythonDataScienceHandbook,gesiscss,github.com,https://github.com/gesiscss/PythonDataScienceHandbook,24,3.17805
7,YoutubeScrapingEmojis,JuKo007,github.com,https://github.com/JuKo007/YoutubeScrapingEmojis,18,2.89037
3,GitHub_traffic_crawler,gesiscss,github.com,https://github.com/gesiscss/GitHub_traffic_crawler,12,2.48491
27,requirements,binder-examples,github.com,https://github.com/binder-examples/requirements,9,2.19722
28,scientific-python-lectures,jrjohansson,github.com,https://github.com/jrjohansson/scientific-python-lectures,9,2.19722
20,ligo-binder,minrk,github.com,https://github.com/minrk/ligo-binder,7,1.94591
11,conda,binder-examples,github.com,https://github.com/binder-examples/conda,7,1.94591
5,RStan-Binder,arnim,github.com,https://github.com/arnim/RStan-Binder,6,1.79176
